In [1]:
import sys
sys.path.append('../')

In [2]:
from f1tenth_gym.f110_env import F110Env
from src.envs.wrapper import F110Wrapper
from f1tenth_gym.maps.map_manager import MapManager, MAP_DICT

map_name = 'Austin'
map_ext = '.png'
speed = 8.0
downsample = 1
use_dynamic_speed = True
a_lat_max = 3
smooth_sigma = 2

map_manager = MapManager(
    map_name=map_name,
    map_ext=map_ext,
    speed=speed,
    downsample=downsample,
    use_dynamic_speed=use_dynamic_speed,
    a_lat_max=a_lat_max,
    smooth_sigma=smooth_sigma
)

vehicle_param = {
    'mu': 1.0489,
    'C_Sf': 4.718,
    'C_Sr': 5.4562,
    'lf': 0.15875,
    'lr': 0.17145,
    'h': 0.074,
    'm': 3.74,
    'I': 0.04712,
    's_min': -0.4,
    's_max': 0.4,
    'sv_min': -3.2,
    'sv_max': 3.2,
    'v_switch': 7.319,
    'a_max': 9.51,
    'v_min': -5.0,
    'v_max': 10.0,
    'width': 0.31,
    'length': 0.58
}

num_beams = 1080
num_agents = 1
## 公式のベース環境
env = F110Env(map=map_manager.map_path, map_ext=map_ext, num_beams=num_beams, num_agents=num_agents, params=vehicle_param)
## 自作のラッパー
env = F110Wrapper(env, map_manager=map_manager)

Loading waypoint type: center


/Users/at/Desktop/project/f1tenth/f1tenth_graph/notebook/../f1tenth_gym/base_classes.py:93: UserWarning: Chosen integrator is RK4. This is different from previous versions of the gym.
  warnings.warn(f"Chosen integrator is RK4. This is different from previous versions of the gym.")


In [4]:
from src.planner.purePursuit import PurePursuitPlanner
wheelbase = 0.33
lookahead = 0.6
gain = 0.2
max_reacquire = 20.0

planner = PurePursuitPlanner(
    wheelbase=wheelbase,
    map_manager=map_manager,
    lookahead=lookahead,
    gain=gain,
    max_reacquire=max_reacquire
)

In [ ]:
import numpy as np
from src.utils.graph import lidar_to_graph
import matplotlib.pyplot as plt
import networkx as nx
from IPython.display import clear_output
import time
max_steps = 3000

obs, info = env.reset()
done = False

for step in range(max_steps):
    actions = []
    for agent_id in range(num_agents):
        steer, speed = planner.plan(obs, id=agent_id)
        action = [steer, speed]
        actions.append(action)

    # グラフの生成
    start = time.time()
    G = lidar_to_graph(scan_data=obs['scans'][0], max_distance=1.0)
    print(f"Graph generation time: {time.time() - start:.4f} seconds")

    # プロットのクリア & 再描画
    # plt.figure(figsize=(8, 6))
    # pos = nx.get_node_attributes(G, 'pos')
    # nx.draw(G, pos, node_size=5, edge_color='lightgray')
    
    # # 表示の更新
    # clear_output(wait=True)
    # plt.show()
    
    next_obs, reward, terminated, truncated, info = env.step(np.array(actions))
    
    if terminated or truncated:
        print(f"Episode finished after {step + 1} timesteps")
        break

    obs = next_obs


NameError: name 'Austin_' is not defined